# Creating SQL Database
Building on our initial exploration, we can ingest and clean the data using the techniques from `exploration.ipynb`.
Rather than using a static CSV file, let's download the most recent CSV files for the `Crisis`, `Use of Force`, and `Crime` datasets. 


In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)


In [2]:
crisis_df = pd.read_csv('https://data.seattle.gov/resource/i2q9-thny.csv?$limit=100000')
crisis_df.replace({'-': None}, inplace=True)

In [3]:
uof_df = pd.read_csv('https://data.seattle.gov/resource/ppi5-g2bj.csv?$limit=100000')

In [4]:
crime_df = pd.read_csv('https://data.seattle.gov/resource/tazs-3rd5.csv?$limit=1500000', parse_dates=['offense_start_datetime', 'report_datetime'])

In [5]:
# import sqlite3
# conn = sqlite3.connect('police-records.db')
# data = crisis_df
# data.to_sql('Crisis', conn, if_exists='replace', index=False)

In [6]:
import sqlite3
conn = sqlite3.connect('police-records.db')
data = crisis_df
data.to_sql('crisis', conn, if_exists='replace', index=False)
data = uof_df
data.to_sql('uof', conn, if_exists='replace', index=False)
data = crime_df
data.to_sql('crime', conn, if_exists='replace', index=False)


1007829

In [7]:
q = '''SELECT COUNT(*)
FROM crisis'''
print(pd.read_sql(q, conn))

   COUNT(*)
0     76247


The crisis dataset website warns us in many places that there is a one to many relationship between the disposition and the event. This means to get accurate counts of the crisis events we need to count the distinct `template_id` values, not rows. 

In [8]:
q = '''SELECT COUNT(DISTINCT template_id) Counts
FROM crisis'''
print(pd.read_sql(q, conn))

   Counts
0   75158


We can check that all our tables in the database look correct:

In [9]:
q = '''SELECT *
FROM crisis
LIMIT 5
OFFSET 10'''
print(pd.read_sql(q, conn))

   template_id        reported_date reported_time        occured_date_time  \
0        43662  2015-05-15T00:00:00      10:22:00  2015-05-15T21:33:28.000   
1        43479  2015-05-15T00:00:00      11:21:00  2015-05-15T18:10:23.000   
2        43946  2015-05-15T00:00:00      10:14:00  2015-05-15T18:47:32.000   
3        43832  2015-05-16T00:00:00      01:24:00  2015-05-16T10:14:07.000   
4        43897  2015-05-16T00:00:00      03:52:00  1900-01-01T00:00:00.000   

  call_type                                initial_call_type  \
0    ONVIEW  HAZ - POTENTIAL THRT TO PHYS SAFETY (NO HAZMAT)   
1       911                 DISTURBANCE, MISCELLANEOUS/OTHER   
2       911     SUICIDE - IP/JO SUICIDAL PERSON AND ATTEMPTS   
3       911                 DISTURBANCE, MISCELLANEOUS/OTHER   
4      None                                             None   

                final_call_type                     disposition  \
0  --CRISIS COMPLAINT - GENERAL  No Action Possible / Necessary   
1  --CRISIS 

Let's see what the most frequent crisis calls are. 

In [10]:
q = '''SELECT *
FROM uof
LIMIT 5
OFFSET 10'''
print(pd.read_sql(q, conn))

                uniqueid  incident_num           incident_type  \
0  2014UOF-0011-1233-223           284  Level 2 - Use of Force   
1  2014UOF-0012-1464-223           271  Level 2 - Use of Force   
2   2014UOF-0013-147-245           292  Level 1 - Use of Force   
3  2014UOF-0014-1202-245           293  Level 1 - Use of Force   
4  2014UOF-0015-1031-169           219  Level 1 - Use of Force   

         occured_date_time precinct   sector beat  officer_id  subject_id  \
0  2014-06-11T02:15:00.000     West     KING   K3        1640         223   
1  2014-06-11T02:15:00.000     West     KING   K3        1141         223   
2  2014-06-17T18:21:00.000     East   GEORGE   G3        1542         245   
3  2014-06-17T18:30:00.000     East   GEORGE   G3        1635         245   
4  2014-05-30T23:23:00.000     East  CHARLIE   C1        1690         169   

                subject_race subject_gender  
0              Not Specified           Male  
1              Not Specified           Male  
2 

In [11]:
q = '''SELECT *
FROM crime
LIMIT 5
OFFSET 10'''
print(pd.read_sql(q, conn))

  report_number   offense_id offense_start_datetime     offense_end_datetime  \
0   2020-044038  12604928711    2020-02-04 20:57:00                     None   
1   2020-043971  12604927228    2019-02-04 00:00:00  2020-02-04T08:00:00.000   
2   2020-043805  12604929082    2020-01-30 19:30:00                     None   
3   2020-043805  12605193820    2020-01-30 19:30:00                     None   
4   2020-043518  12604909238    2020-02-04 07:30:00  2020-02-04T10:00:00.000   

       report_datetime group_a_b crime_against_category  offense_parent_group  \
0  2020-02-04 21:20:35         A               PROPERTY         LARCENY-THEFT   
1  2020-02-04 21:18:52         A               PROPERTY        FRAUD OFFENSES   
2  2020-02-04 21:14:00         A               PROPERTY         LARCENY-THEFT   
3  2020-02-04 21:14:00         A                 PERSON  KIDNAPPING/ABDUCTION   
4  2020-02-04 20:59:01         A               PROPERTY         LARCENY-THEFT   

                    offense offe

# Querying the database
Now that our database is created, we can run various queries to try to extract conclusions.

What are the most common crisis call types?

In [12]:
q = '''SELECT initial_call_type, count(*) Counts 
FROM crisis
GROUP BY initial_call_type
ORDER BY Counts DESC
LIMIT 25
'''
print(pd.read_sql(q, conn))

                                  initial_call_type  Counts
0             PERSON IN BEHAVIORAL/EMOTIONAL CRISIS   13753
1      SUICIDE - IP/JO SUICIDAL PERSON AND ATTEMPTS   13349
2                  DISTURBANCE, MISCELLANEOUS/OTHER    6564
3                                              None    5614
4            SUSPICIOUS PERSON, VEHICLE OR INCIDENT    2573
5                           SERVICE - WELFARE CHECK    2254
6             SUICIDE, SUICIDAL PERSON AND ATTEMPTS    2188
7                                          TRESPASS    2085
8                  DIST - IP/JO - DV DIST - NO ASLT    2000
9     THREATS (INCLS IN-PERSON/BY PHONE/IN WRITING)    1810
10   ASLT - IP/JO - WITH OR W/O WPNS (NO SHOOTINGS)    1633
11           SFD - ASSIST ON FIRE OR MEDIC RESPONSE    1627
12                              DIST - DV - NO ASLT    1623
13            ASSIST OTHER AGENCY - ROUTINE SERVICE    1481
14  HAZ - POTENTIAL THRT TO PHYS SAFETY (NO HAZMAT)    1255
15        ASLT - WITH OR W/O WEAPONS (NO

How many crisis calls each year had a CIT officer requested, but not arrived?

In [13]:
q = '''SELECT strftime('%Y', reported_date) Year, COUNT(DISTINCT template_id) "No CIT" 
FROM crisis
WHERE cit_officer_requested = "Y" AND
cit_officer_arrived = "N"
GROUP BY Year
ORDER BY Year
'''
print(pd.read_sql(q, conn))

   Year  No CIT
0  2015     661
1  2016     796
2  2017     716
3  2018     774
4  2019     198


Let's compare the crisis counts per beat with use of force per beat. 

In [14]:
q = '''SELECT beat, "Crisis Count", "Use of Force Count", "Crisis Count"/1.0/"Use of Force Count" Ratio FROM (
SELECT uof.beat, COUNT(DISTINCT uniqueid) "Use of Force Count", COUNT(DISTINCT template_id) "Crisis Count"
FROM uof
JOIN crisis ON uof.beat = crisis.beat
WHERE uof.beat != "-" AND uof.beat != "99"
GROUP BY uof.beat
) sub
ORDER BY "Crisis Count" DESC
'''
print(pd.read_sql(q, conn))

   beat  Crisis Count  Use of Force Count      Ratio
0    K3          2621                 514   5.099222
1    E3          2520                 265   9.509434
2    E1          2439                 365   6.682192
3    K2          2395                 521   4.596929
4    K1          2036                 492   4.138211
5    D2          1942                 472   4.114407
6    G1          1876                 229   8.192140
7    N3          1841                 469   3.925373
8    D3          1839                 244   7.536885
9    D1          1825                 357   5.112045
10   E2          1693                1117   1.515667
11   Q3          1638                 239   6.853556
12   R2          1525                 420   3.630952
13   B1          1519                 210   7.233333
14   U2          1478                 297   4.976431
15   M1          1435                 274   5.237226
16   N2          1433                 321   4.464174
17   L3          1403                 205   6.

What are the most common crimes commited in Northgate?

In [15]:
q = '''SELECT offense_parent_group "Crime Type", COUNT(DISTINCT offense_id) Count
FROM crime 
WHERE mcpp = "NORTHGATE"
GROUP BY "Crime Type"
ORDER BY Count DESC
LIMIT 15
'''
print(pd.read_sql(q, conn))

                                  Crime Type  Count
0                              LARCENY-THEFT  23341
1                           ASSAULT OFFENSES   6984
2                 BURGLARY/BREAKING&ENTERING   5307
3   DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY   4275
4                        MOTOR VEHICLE THEFT   3777
5                             FRAUD OFFENSES   3335
6                  TRESPASS OF REAL PROPERTY   1785
7                      PROSTITUTION OFFENSES   1712
8                                    ROBBERY   1420
9                     DRUG/NARCOTIC OFFENSES   1295
10               DRIVING UNDER THE INFLUENCE    891
11               FAMILY OFFENSES, NONVIOLENT    584
12                  STOLEN PROPERTY OFFENSES    465
13                     WEAPON LAW VIOLATIONS    419
14                    COUNTERFEITING/FORGERY    415
